# 使用 keyword index simple 方式

使用 obsidian 作为数据源。

评估 keyword table index `simple` 方式，是否可以作为 embedding 方式的补充。

初步结论：

- 效果不好，查询结果为空
- llamaindex的简单分词无法支持中文

## 全局设置

In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 730 ms, sys: 47.9 ms, total: 778 ms
Wall time: 778 ms


## 加载文档

In [3]:
%%time

from llama_index.readers.obsidian import ObsidianReader


documents = ObsidianReader(
    "/models/obsidian/kb/知识库"
).load_data() 

len(documents)

CPU times: user 0 ns, sys: 2.37 ms, total: 2.37 ms
Wall time: 2.2 ms


21

## 创建索引

In [4]:
%%time

from llama_index.core import SimpleKeywordTableIndex

index = SimpleKeywordTableIndex.from_documents(
    documents
)

CPU times: user 227 ms, sys: 19.9 ms, total: 247 ms
Wall time: 246 ms


## 查询

In [5]:
%%time

query_engine = index.as_query_engine(
    streaming=True
)
streaming_response = query_engine.query("docker容器在后台执行的命令")
streaming_response.print_response_stream()

Empty ResponseCPU times: user 2.34 ms, sys: 271 µs, total: 2.61 ms
Wall time: 2.17 ms
